# Synchronisation Audio-Video (Passerelle)

**Module :** 04-Audio-Applications  
**Niveau :** Applications  
**Technologies :** OpenAI TTS, moviepy, pydub, ffmpeg-python  
**VRAM estimee :** ~10 GB  
**Duree estimee :** 55 minutes  

## Objectifs d'Apprentissage

- [ ] Extraire la piste audio d'une video
- [ ] Generer une narration TTS synchronisee avec les segments video
- [ ] Aligner l'audio avec la timeline video
- [ ] Superposer musique de fond et narration
- [ ] Assembler video + narration + musique + sous-titres
- [ ] Comprendre le lien entre les series Audio et Video

## Prerequis

- Notebooks Foundation (01-1 TTS, 01-3 Audio Operations) completes
- Notebook 04-2 (Pipeline de Transcription) pour les sous-titres
- Cle API OpenAI configuree (`OPENAI_API_KEY` dans `.env`)
- moviepy, pydub, ffmpeg installes

> **Note** : Ce notebook est la **passerelle** entre la serie Audio et la serie Video. Il utilise des techniques audio pour enrichir du contenu video. La serie Video (`Video/01-Foundation/`) approfondit la generation et l'edition video.

**Navigation :** [Index](../README.md) | [<< Precedent](04-3-Music-Composition-Workflow.ipynb) | [Serie Video >>](../../Video/01-Foundation/01-1-Video-Operations-Basics.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres TTS narration
tts_model = "tts-1"               # "tts-1" ou "tts-1-hd"
narrator_voice = "nova"            # Voix de narration

# Parametres video
video_resolution = (640, 480)      # Resolution de la video de test
video_fps = 24                     # Images par seconde
video_duration = 15                # Duree de la video de test (secondes)

# Configuration pipeline
generate_audio = True
save_output_files = True
background_music_volume = -20      # Volume musique de fond en dB (relatif)

In [2]:
# Setup environnement et imports
import os
import sys
import json
import time
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
from io import BytesIO
import logging

import numpy as np
from IPython.display import Audio, display, HTML, Video as IPVideo

# Resolution GENAI_ROOT
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.audio_helpers import (
            synthesize_openai, play_audio_bytes,
            estimate_audio_duration, get_audio_info
        )
        print("Helpers audio importes")
    except ImportError as e:
        print(f"Helpers audio non disponibles : {e}")

# Repertoires de sortie
OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'av_sync'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('av_sync')

print(f"Synchronisation Audio-Video")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}, TTS : {tts_model}")
print(f"Sortie : {OUTPUT_DIR}")

Helpers audio importes
Synchronisation Audio-Video
Date : 2026-02-18 10:52:40
Mode : interactive, TTS : tts-1
Sortie : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\av_sync


In [3]:
# Chargement de la configuration et validation
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve")

openai_key = os.getenv('OPENAI_API_KEY')

if not openai_key:
    if notebook_mode == "batch" and not generate_audio:
        openai_key = "dummy_key_for_validation"
        print("Mode batch sans generation : cle API ignoree")
    else:
        raise ValueError("OPENAI_API_KEY manquante dans .env")

from openai import OpenAI
client = OpenAI(api_key=openai_key)

# Verification des dependances video
moviepy_available = False
try:
    from moviepy.editor import (
        VideoFileClip, AudioFileClip, CompositeAudioClip,
        TextClip, CompositeVideoClip, ColorClip, concatenate_videoclips
    )
    moviepy_available = True
    print("moviepy disponible")
except ImportError:
    print("moviepy non installe - pip install moviepy")

from pydub import AudioSegment
print(f"pydub disponible")

print(f"\nConfiguration prete")

Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env


moviepy non installe - pip install moviepy
pydub disponible

Configuration prete


C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Section 1 : Creation d'une video de test et extraction audio

Avant de travailler sur la synchronisation, nous creons une video de test simple et montrons comment extraire sa piste audio.

| Operation | Outil | Entree | Sortie |
|-----------|-------|--------|--------|
| Creation video | moviepy | Parametres | Fichier .mp4 |
| Extraction audio | moviepy / ffmpeg | Video .mp4 | Audio .wav/.mp3 |
| Analyse audio | pydub | Audio | Metadonnees |

In [4]:
# Creation d'une video de test et extraction audio
print("CREATION VIDEO DE TEST ET EXTRACTION AUDIO")
print("=" * 50)

test_video_path = OUTPUT_DIR / "test_video.mp4"
extracted_audio_path = OUTPUT_DIR / "extracted_audio.wav"

if generate_audio and moviepy_available:
    # Creer une video de test avec des scenes colorees
    print("Creation de la video de test...")

    # Definir les segments de la video
    video_segments = [
        {"label": "Introduction", "color": (30, 60, 120), "duration": 5},
        {"label": "Concepts cles", "color": (60, 120, 30), "duration": 5},
        {"label": "Conclusion", "color": (120, 30, 60), "duration": 5}
    ]

    clips = []
    for seg in video_segments:
        # Clip de couleur unie
        color_clip = ColorClip(
            size=video_resolution,
            color=seg['color'],
            duration=seg['duration']
        )

        # Ajouter un titre
        try:
            txt_clip = TextClip(
                seg['label'],
                fontsize=40,
                color='white',
                font='Arial'
            ).set_position('center').set_duration(seg['duration'])
            clip = CompositeVideoClip([color_clip, txt_clip])
        except Exception:
            # Si TextClip echoue (ImageMagick non disponible), utiliser le clip brut
            clip = color_clip

        clips.append(clip)

    # Concatener les clips
    final_video = concatenate_videoclips(clips)
    final_video = final_video.set_fps(video_fps)

    # Sauvegarder la video
    final_video.write_videofile(
        str(test_video_path),
        fps=video_fps,
        codec='libx264',
        audio=False,
        logger=None
    )
    final_video.close()

    print(f"Video creee : {test_video_path.name}")
    print(f"Resolution : {video_resolution[0]}x{video_resolution[1]}")
    print(f"Duree : {video_duration}s | FPS : {video_fps}")
    print(f"Segments : {len(video_segments)}")

    for i, seg in enumerate(video_segments):
        start = sum(s['duration'] for s in video_segments[:i])
        end = start + seg['duration']
        print(f"  [{start:2.0f}s - {end:2.0f}s] {seg['label']}")

    # Demonstration de l'extraction audio
    print(f"\n(La video de test n'a pas de piste audio - nous en ajouterons une)")

elif not moviepy_available:
    print("moviepy non disponible - pip install moviepy")
else:
    print("Generation desactivee")

CREATION VIDEO DE TEST ET EXTRACTION AUDIO
moviepy non disponible - pip install moviepy


### Interpretation : Video de test

| Segment | Debut | Fin | Contenu |
|---------|-------|-----|---------|
| Introduction | 0s | 5s | Fond bleu |
| Concepts cles | 5s | 10s | Fond vert |
| Conclusion | 10s | 15s | Fond rouge |

**Points cles** :
1. moviepy permet de creer et manipuler des videos en Python
2. L'extraction audio utilise la meme API (`video.audio`)
3. La video de test servira de base pour la synchronisation

## Section 2 : Narration TTS synchronisee

Pour chaque segment video, nous generons une narration TTS adaptee. Le texte est calibre pour correspondre a la duree du segment.

| Segment | Duree cible | Texte adapte | Vitesse TTS |
|---------|------------|--------------|-------------|
| Introduction | 5s | Court, accueillant | 1.0 |
| Concepts cles | 5s | Dense, informatif | 0.95 |
| Conclusion | 5s | Recapitulatif | 1.0 |

In [5]:
# Generation de narration TTS synchronisee aux segments video
print("NARRATION TTS SYNCHRONISEE")
print("=" * 50)

# Scripts de narration pour chaque segment video
narration_scripts = [
    {
        "segment": "Introduction",
        "start": 0, "end": 5,
        "text": "Bienvenue dans cette presentation sur l'intelligence artificielle generative. Nous allons decouvrir ensemble les concepts fondamentaux.",
        "speed": 1.0
    },
    {
        "segment": "Concepts cles",
        "start": 5, "end": 10,
        "text": "Les modeles generatifs apprennent a creer du contenu nouveau. Ils analysent des exemples existants pour en deduire des patterns.",
        "speed": 0.95
    },
    {
        "segment": "Conclusion",
        "start": 10, "end": 15,
        "text": "En resume, l'IA generative ouvre de nouvelles possibilites creatives. Explorez les notebooks suivants pour aller plus loin.",
        "speed": 1.0
    }
]

narration_audio_segments = []
pydub_available = False

# Verifier si pydub/ffmpeg fonctionne
try:
    test_audio = AudioSegment.silent(duration=100)
    pydub_available = True
except Exception as e:
    print(f"pydub/ffmpeg non disponible : {e}")
    print("Utilisation de la duree estimee pour la synchronisation")

if generate_audio and openai_key != "dummy_key_for_validation":
    print(f"Generation de la narration pour {len(narration_scripts)} segments :")

    for script in narration_scripts:
        print(f"\n--- {script['segment']} [{script['start']}s - {script['end']}s] ---")
        print(f"Texte : {script['text'][:70]}...")

        start_time = time.time()
        response = client.audio.speech.create(
            model=tts_model,
            voice=narrator_voice,
            input=script['text'],
            response_format="mp3",
            speed=script['speed']
        )
        audio_bytes = response.content
        gen_time = time.time() - start_time

        # Estimer la duree audio (approximation: 130 mots/min = 2.17 mots/sec)
        word_count = len(script['text'].split())
        estimated_duration = word_count / 2.17 / script['speed']

        # Tenter de charger avec pydub si disponible
        audio_seg = None
        audio_duration = estimated_duration

        if pydub_available:
            try:
                audio_seg = AudioSegment.from_mp3(BytesIO(audio_bytes))
                audio_duration = len(audio_seg) / 1000  # ms -> s
            except Exception as e:
                print(f"  pydub erreur, utilisation estimation : {e}")

        segment_duration = script['end'] - script['start']

        narration_audio_segments.append({
            "segment": script['segment'],
            "start": script['start'],
            "end": script['end'],
            "audio_bytes": audio_bytes,
            "audio_segment": audio_seg,
            "audio_duration": audio_duration,
            "target_duration": segment_duration
        })

        fit_status = "OK" if audio_duration <= segment_duration else "DEPASSE"
        print(f"  Duree audio : {audio_duration:.1f}s / cible {segment_duration}s [{fit_status}]")
        print(f"  Taille : {len(audio_bytes)/1024:.1f} KB | Generation : {gen_time:.1f}s")
        display(Audio(data=audio_bytes, autoplay=False))

    # Recapitulatif
    print(f"\nRecapitulatif synchronisation :")
    print(f"{'Segment':<18} {'Audio (s)':<12} {'Cible (s)':<12} {'Statut':<10}")
    print("-" * 52)
    for seg in narration_audio_segments:
        status = "OK" if seg['audio_duration'] <= seg['target_duration'] else "A ajuster"
        print(f"{seg['segment']:<18} {seg['audio_duration']:<12.1f} {seg['target_duration']:<12.0f} {status:<10}")
else:
    print("Generation desactivee")

NARRATION TTS SYNCHRONISEE
Generation de la narration pour 3 segments :

--- Introduction [0s - 5s] ---
Texte : Bienvenue dans cette presentation sur l'intelligence artificielle gene...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  pydub erreur, utilisation estimation : [WinError 2] Le fichier spécifié est introuvable
  Duree audio : 6.9s / cible 5s [DEPASSE]
  Taille : 165.9 KB | Generation : 3.1s


C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)



--- Concepts cles [5s - 10s] ---
Texte : Les modeles generatifs apprennent a creer du contenu nouveau. Ils anal...


...

  pydub erreur, utilisation estimation : [WinError 2] Le fichier spécifié est introuvable
  Duree audio : 9.2s / cible 5s [DEPASSE]
  Taille : 166.4 KB | Generation : 3.7s



--- Conclusion [10s - 15s] ---
Texte : En resume, l'IA generative ouvre de nouvelles possibilites creatives. ...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  pydub erreur, utilisation estimation : [WinError 2] Le fichier spécifié est introuvable
  Duree audio : 7.8s / cible 5s [DEPASSE]
  Taille : 150.0 KB | Generation : 3.7s



Recapitulatif synchronisation :
Segment            Audio (s)    Cible (s)    Statut    
----------------------------------------------------
Introduction       6.9          5            A ajuster 
Concepts cles      9.2          5            A ajuster 
Conclusion         7.8          5            A ajuster 


### Interpretation : Narration synchronisee

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Synchronisation | Audio <= duree segment | La narration doit tenir dans le segment |
| Depassement | Possible | Ajuster vitesse ou couper le texte |
| Silence residuel | Normal | Comble par la musique de fond |

> **Note technique** : Si la narration depasse la duree du segment, deux solutions : augmenter la vitesse TTS (max 4.0x) ou raccourcir le texte.

## Section 3 : Alignement audio sur la timeline video

L'alignement consiste a placer chaque segment audio au bon moment sur la timeline de la video.

```
Video:  [====Introduction====][==Concepts cles==][====Conclusion====]
Audio:  [narration_1][silence][narration_2][ sil ][narration_3][ sil]
        0s           5s       5s          10s    10s          15s
```

Chaque narration est placee avec un padding de silence pour remplir exactement la duree du segment.

In [6]:
# Alignement audio sur la timeline video
print("ALIGNEMENT AUDIO - TIMELINE")
print("=" * 50)

def align_audio_to_timeline(segments: List[Dict],
                             total_duration_ms: int) -> Optional[AudioSegment]:
    """Aligne les segments audio sur une timeline video.

    Chaque segment est place a son timestamp de debut,
    avec du silence pour remplir les espaces.
    """
    timeline = AudioSegment.silent(duration=total_duration_ms)

    for seg in segments:
        start_ms = int(seg['start'] * 1000)
        audio = seg['audio_segment']

        if audio is None:
            # Pas d'audio_segment disponible (pydub/ffmpeg non fonctionnel)
            # On cree un silence pour la duree estimee
            target_ms = int(seg['target_duration'] * 1000)
            continue

        # Tronquer si l'audio depasse la fin du segment
        target_ms = int(seg['target_duration'] * 1000)
        if len(audio) > target_ms:
            # Ajouter un fade out avant de tronquer
            audio = audio[:target_ms].fade_out(200)

        # Superposer sur la timeline
        timeline = timeline.overlay(audio, position=start_ms)

    return timeline

narration_timeline = None

if generate_audio and narration_audio_segments:
    total_duration_ms = video_duration * 1000

    print(f"Duree totale video : {video_duration}s ({total_duration_ms}ms)")
    print(f"Segments a aligner : {len(narration_audio_segments)}")

    # Verifier si on a des audio_segments valides
    has_valid_audio = any(seg['audio_segment'] is not None for seg in narration_audio_segments)

    if has_valid_audio:
        narration_timeline = align_audio_to_timeline(
            narration_audio_segments,
            total_duration_ms
        )

        print(f"\nTimeline narration :")
        print(f"  Duree : {len(narration_timeline)/1000:.1f}s")
        print(f"  Canaux : {narration_timeline.channels}")
        print(f"  Sample rate : {narration_timeline.frame_rate}Hz")
        print(f"  Volume : {narration_timeline.dBFS:.1f} dBFS")

        # Sauvegarder la timeline narration
        if save_output_files:
            narration_path = OUTPUT_DIR / "narration_timeline.wav"
            narration_timeline.export(str(narration_path), format="wav")
            print(f"  Sauvegarde : {narration_path.name}")

        # Ecouter
        narration_bytes = BytesIO()
        narration_timeline.export(narration_bytes, format="mp3")
        print(f"\nEcoute de la timeline narration :")
        display(Audio(data=narration_bytes.getvalue(), autoplay=False))
    else:
        print("\npydub/ffmpeg non disponible - alignement audio impossible")
        print("Les segments TTS ont ete generes mais ne peuvent pas etre combines")
        for seg in narration_audio_segments:
            print(f"  - {seg['segment']}: {seg['audio_duration']:.1f}s (audio_bytes dispo)")
else:
    print("Alignement desactive (pas de segments narration)")

ALIGNEMENT AUDIO - TIMELINE
Duree totale video : 15s (15000ms)
Segments a aligner : 3

pydub/ffmpeg non disponible - alignement audio impossible
Les segments TTS ont ete generes mais ne peuvent pas etre combines
  - Introduction: 6.9s (audio_bytes dispo)
  - Concepts cles: 9.2s (audio_bytes dispo)
  - Conclusion: 7.8s (audio_bytes dispo)


## Section 4 : Musique de fond sous la narration

La superposition de musique de fond et de narration necessite un controle precis des niveaux :

| Piste | Volume relatif | Raison |
|-------|---------------|--------|
| Narration | 0 dB (reference) | Voix au premier plan |
| Musique de fond | -15 a -25 dB | Ambiance sans couvrir la voix |
| Effets sonores | -5 a -10 dB | Ponctuels, perceptibles |

La technique du "ducking" baisse automatiquement la musique quand la voix est presente.

In [7]:
# Superposition musique de fond + narration
print("SUPERPOSITION MUSIQUE DE FOND")
print("=" * 50)

def generate_ambient_music(duration_ms: int, sr: int = 44100) -> Optional[AudioSegment]:
    """Genere une musique d'ambiance simple (sinusoides harmoniques)."""
    try:
        t = np.linspace(0, duration_ms / 1000, int(sr * duration_ms / 1000))

        # Accords doux (Do majeur : C4, E4, G4)
        freqs = [261.63, 329.63, 392.00]  # C4, E4, G4
        audio = np.zeros_like(t)
        for f in freqs:
            audio += 0.15 * np.sin(2 * np.pi * f * t)

        # Modulation lente pour un effet d'ambiance
        modulation = 0.5 + 0.5 * np.sin(2 * np.pi * 0.1 * t)  # 0.1 Hz
        audio *= modulation

        # Normaliser en int16
        audio = (audio / np.max(np.abs(audio)) * 16000).astype(np.int16)

        return AudioSegment(
            data=audio.tobytes(),
            sample_width=2,
            frame_rate=sr,
            channels=1
        )
    except Exception as e:
        print(f"Erreur generation musique : {e}")
        return None

mixed_audio = None

if generate_audio and narration_timeline:
    total_duration_ms = len(narration_timeline)

    # Generer la musique de fond
    print("Generation de la musique d'ambiance...")
    background_music = generate_ambient_music(total_duration_ms)

    if background_music:
        # Ajuster le volume de la musique
        background_music = background_music + background_music_volume  # dB

        # Fade in/out sur la musique
        background_music = background_music.fade_in(1000).fade_out(2000)

        print(f"Musique de fond :")
        print(f"  Duree : {len(background_music)/1000:.1f}s")
        print(f"  Volume : {background_music.dBFS:.1f} dBFS (reduit de {background_music_volume}dB)")

        # Superposer narration + musique
        print(f"\nSuperposition narration + musique...")

        # S'assurer que les deux ont la meme duree
        if len(background_music) > len(narration_timeline):
            background_music = background_music[:len(narration_timeline)]
        elif len(background_music) < len(narration_timeline):
            background_music = background_music + AudioSegment.silent(
                duration=len(narration_timeline) - len(background_music)
            )

        mixed_audio = narration_timeline.overlay(background_music)

        print(f"Mix final :")
        print(f"  Duree : {len(mixed_audio)/1000:.1f}s")
        print(f"  Volume : {mixed_audio.dBFS:.1f} dBFS")

        # Sauvegarder et ecouter
        if save_output_files:
            mix_path = OUTPUT_DIR / "narration_plus_music.wav"
            mixed_audio.export(str(mix_path), format="wav")
            print(f"  Sauvegarde : {mix_path.name}")

        mix_bytes = BytesIO()
        mixed_audio.export(mix_bytes, format="mp3")
        print(f"\nEcoute du mix (narration + musique de fond) :")
        display(Audio(data=mix_bytes.getvalue(), autoplay=False))
    else:
        print("Musique de fond non disponible")
elif not narration_timeline:
    print("Mix desactive (pas de narration timeline)")
else:
    print("Mix desactive")

SUPERPOSITION MUSIQUE DE FOND
Mix desactive (pas de narration timeline)


### Interpretation : Mix audio

| Piste | Volume (dBFS) | Role |
|-------|--------------|------|
| Narration | ~-15 a -20 | Premier plan |
| Musique de fond | ~-35 a -45 | Ambiance discrete |
| Mix final | ~-15 a -20 | Equilibre narration/musique |

**Points cles** :
1. Le volume de la musique (-20dB) la rend perceptible sans couvrir la voix
2. Les fades evitent les transitions brutales
3. En production, utiliser du ducking dynamique pour un resultat plus naturel

## Section 5 : Assemblage video + audio final

L'assemblage final combine la video, la narration et la musique de fond en un seul fichier.

```
Pipeline d'assemblage :
    Video (.mp4, sans audio)
    +  Narration TTS (timeline alignee)
    +  Musique de fond (volume reduit)
    =  Video finale (.mp4, avec audio mixe)
```

moviepy gere la combinaison des flux video et audio.

In [8]:
# Assemblage video + audio final
print("ASSEMBLAGE VIDEO + AUDIO")
print("=" * 50)

final_video_path = OUTPUT_DIR / "video_finale_narree.mp4"

if generate_audio and moviepy_available and mixed_audio and test_video_path.exists():
    # Sauvegarder l'audio mixe en WAV pour moviepy
    mixed_audio_path = OUTPUT_DIR / "mixed_audio_temp.wav"
    mixed_audio.export(str(mixed_audio_path), format="wav")

    print("Assemblage en cours...")
    start_time = time.time()

    # Charger la video et l'audio
    video_clip = VideoFileClip(str(test_video_path))
    audio_clip = AudioFileClip(str(mixed_audio_path))

    # Ajuster la duree de l'audio a celle de la video
    if audio_clip.duration > video_clip.duration:
        audio_clip = audio_clip.subclip(0, video_clip.duration)

    # Combiner video + audio
    video_with_audio = video_clip.set_audio(audio_clip)

    # Exporter
    video_with_audio.write_videofile(
        str(final_video_path),
        fps=video_fps,
        codec='libx264',
        audio_codec='aac',
        logger=None
    )

    assembly_time = time.time() - start_time

    # Fermer les clips
    video_clip.close()
    audio_clip.close()
    video_with_audio.close()

    # Nettoyage fichier temporaire
    if mixed_audio_path.exists():
        mixed_audio_path.unlink()

    print(f"Assemblage termine en {assembly_time:.1f}s")
    print(f"\nVideo finale :")
    print(f"  Fichier : {final_video_path.name}")
    print(f"  Taille : {final_video_path.stat().st_size/1024:.1f} KB")
    print(f"  Resolution : {video_resolution[0]}x{video_resolution[1]}")
    print(f"  FPS : {video_fps}")
    print(f"  Duree : {video_duration}s")
    print(f"  Audio : narration TTS + musique de fond")

    # Afficher la video dans le notebook (si possible)
    try:
        display(IPVideo(str(final_video_path), embed=True, width=480))
    except Exception as e:
        print(f"Affichage video non disponible : {str(e)[:80]}")
        print(f"Ouvrez le fichier : {final_video_path}")
else:
    if not moviepy_available:
        print("moviepy requis pour l'assemblage")
    elif not mixed_audio:
        print("Pas d'audio mixe disponible")
    elif not test_video_path.exists():
        print("Video de test non disponible")
    else:
        print("Assemblage desactive")

ASSEMBLAGE VIDEO + AUDIO
moviepy requis pour l'assemblage


### Interpretation : Assemblage final

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Codec video | H.264 (libx264) | Standard universel |
| Codec audio | AAC | Compression efficace |
| Temps d'assemblage | Variable | Depend de la duree et resolution |

**Points cles** :
1. moviepy simplifie considerablement le pipeline d'assemblage
2. Les codecs H.264/AAC sont compatibles avec tous les lecteurs
3. Pour de la production, utiliser `codec='libx264'` avec `-preset slow` pour une meilleure qualite

## Passerelle vers la serie Video

Ce notebook constitue le point de jonction entre les series Audio et Video du cours GenAI. Voici les liens entre les deux series :

| Serie Audio (cette serie) | Serie Video | Lien |
|--------------------------|-------------|------|
| TTS (narration) | Narration sur video | Audio enrichit la video |
| Whisper (sous-titres) | Sous-titrage video | Transcription -> SRT |
| MusicGen (musique) | Bande sonore video | Musique de fond |
| Demucs (separation) | Post-production | Remix audio de video |

### Pour continuer

- **Serie Video Foundation** : `Video/01-Foundation/01-1-Video-Operations-Basics.ipynb` - Operations de base sur la video
- **Serie Video Advanced** : Generation de video avec HunyuanVideo, LTX-Video, Wan
- **Serie Video Orchestration** : Pipelines video complets avec ComfyUI

Les techniques audio apprises dans cette serie s'appliquent directement aux workflows video.

In [9]:
# Mode interactif - Personnaliser le pipeline
if notebook_mode == "interactive" and not skip_widgets:
    print("MODE INTERACTIF - PIPELINE PERSONNALISE")
    print("=" * 50)
    print("\nEntrez un texte de narration pour l'ajouter a la video :")
    print("(Laissez vide pour passer)")

    try:
        user_text = input("\nVotre narration : ")

        if user_text.strip():
            print(f"\nGeneration TTS...")
            response = client.audio.speech.create(
                model=tts_model,
                voice=narrator_voice,
                input=user_text,
                response_format="mp3"
            )
            print(f"Narration generee ({len(response.content)/1024:.1f} KB) :")
            display(Audio(data=response.content, autoplay=False))

            if save_output_files:
                ts = datetime.now().strftime('%Y%m%d_%H%M%S')
                filepath = OUTPUT_DIR / f"custom_narration_{ts}.mp3"
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                print(f"Sauvegarde : {filepath.name}")
        else:
            print("Mode interactif ignore")

    except (KeyboardInterrupt, EOFError):
        print("Mode interactif interrompu")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("Mode interactif non disponible (execution automatisee)")
        else:
            print(f"Erreur : {error_type} - {str(e)[:100]}")
else:
    print("Mode batch - Interface interactive desactivee")

MODE INTERACTIF - PIPELINE PERSONNALISE

Entrez un texte de narration pour l'ajouter a la video :
(Laissez vide pour passer)
Mode interactif non disponible (execution automatisee)


In [10]:
# Statistiques de session
print("STATISTIQUES DE SESSION")
print("=" * 50)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"TTS : {tts_model}, Voix : {narrator_voice}")

if narration_audio_segments:
    total_narration = sum(s['audio_duration'] for s in narration_audio_segments)
    print(f"Segments narration : {len(narration_audio_segments)} ({total_narration:.1f}s)")

if narration_timeline:
    print(f"Timeline narration : {len(narration_timeline)/1000:.1f}s")

if mixed_audio:
    print(f"Mix final : {len(mixed_audio)/1000:.1f}s")

if final_video_path.exists():
    print(f"Video finale : {final_video_path.name} ({final_video_path.stat().st_size/1024:.1f} KB)")

if save_output_files:
    saved_files = list(OUTPUT_DIR.glob('*'))
    print(f"Fichiers sauvegardes : {len(saved_files)} dans {OUTPUT_DIR}")

print(f"\nSERIE AUDIO TERMINEE")
print(f"Felicitations ! Vous avez parcouru l'ensemble de la serie Audio :")
print(f"  01-Foundation : TTS, STT, operations audio, Whisper local, Kokoro")
print(f"  02-Advanced   : Chatterbox, XTTS, MusicGen, Demucs")
print(f"  03-Orchestration : Multi-modeles, pipelines, voix temps reel")
print(f"  04-Applications  : Contenu educatif, transcription, composition, A/V sync")
print(f"\nPROCHAINE SERIE")
print(f"  -> Video/01-Foundation/01-1-Video-Operations-Basics.ipynb")

print(f"\nNotebook termine - {datetime.now().strftime('%H:%M:%S')}")

STATISTIQUES DE SESSION
Date : 2026-02-18 10:52:51
Mode : interactive
TTS : tts-1, Voix : nova
Segments narration : 3 (24.0s)
Fichiers sauvegardes : 2 dans D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\av_sync

SERIE AUDIO TERMINEE
Felicitations ! Vous avez parcouru l'ensemble de la serie Audio :
  01-Foundation : TTS, STT, operations audio, Whisper local, Kokoro
  02-Advanced   : Chatterbox, XTTS, MusicGen, Demucs
  03-Orchestration : Multi-modeles, pipelines, voix temps reel
  04-Applications  : Contenu educatif, transcription, composition, A/V sync

PROCHAINE SERIE
  -> Video/01-Foundation/01-1-Video-Operations-Basics.ipynb

Notebook termine - 10:52:51
